# Evaluate Model

### Load Packages

In [65]:
import os
import time
import re

### Setup

In [5]:
SETUP_TENSORFLOW = True

WORKSPACE_PATH = "workspace"
DATA_PATH = os.path.join(WORKSPACE_PATH, "data")
PRE_TRAINED_MODELS_PATH = os.path.join(WORKSPACE_PATH, "pre_trained_models")

MODELS_PATH = os.path.join(WORKSPACE_PATH, "models")
BASE_MODEL_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711"
MODEL_NAME = "efficientdet_d1_coco17_tpu-32"

PIPELINE_CONFIG_PATH = os.path.join(PRE_TRAINED_MODELS_PATH, MODEL_NAME, "pipeline.config")

NUM_CLASSES = 10

EPOCHS = 100
BATCH_SIZE = 4
LEARNING_RATE = 1e-2
USE_AUGMENTATION = False
COMBINATION_NAME = f"epochs_{EPOCHS}-batch_size_{BATCH_SIZE}-learning_rate_{LEARNING_RATE}-aug_{USE_AUGMENTATION}"

In [57]:
checkpoints = [int(file_name.split('.')[0].split("-")[1]) for file_name in os.listdir(os.path.join(MODELS_PATH, MODEL_NAME, COMBINATION_NAME)) if file_name.startswith("ckpt") and not file_name.endswith("index")]

checkpoints[:10]

TypeError: 'NoneType' object is not subscriptable

In [59]:
#!rm -rf {MODELS_PATH}/{MODEL_NAME}/{COMBINATION_NAME}/eval

In [60]:
COMBINATION_NAME

'epochs_100-batch_size_4-learning_rate_0.01-aug_False'

In [61]:
!python workspace/model_main_tf2.py \
    --pipeline_config_path=workspace/models/{MODEL_NAME}/{COMBINATION_NAME}/pipeline.config \
    --model_dir=workspace/models/{MODEL_NAME}/{COMBINATION_NAME} \
    --checkpoint_dir=workspace/models/{MODEL_NAME}/{COMBINATION_NAME} \
    --eval_timeout=0 \
    --alsologtostderr

/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase3GetEPNS_15OpKern

### Download model files
1. Zip all files
2. Download zip

In [62]:
start_time = time.time()
!tar cvfz workspace/models/{MODEL_NAME}/{COMBINATION_NAME}/allfiles.tar.gz workspace/models/{MODEL_NAME}/{COMBINATION_NAME}

end_time = time.time()
elapsed_time = (end_time - start_time) / 60

print(f"Elapsed time: {round(elapsed_time, 2)} minutes")

workspace/models/efficientdet_d1_coco17_tpu-32/epochs_100-batch_size_4-learning_rate_0.01-aug_False/
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_100-batch_size_4-learning_rate_0.01-aug_False/ckpt-60.data-00000-of-00001
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_100-batch_size_4-learning_rate_0.01-aug_False/ckpt-189.data-00000-of-00001
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_100-batch_size_4-learning_rate_0.01-aug_False/ckpt-194.data-00000-of-00001
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_100-batch_size_4-learning_rate_0.01-aug_False/ckpt-35.data-00000-of-00001
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_100-batch_size_4-learning_rate_0.01-aug_False/ckpt-118.index
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_100-batch_size_4-learning_rate_0.01-aug_False/ckpt-177.index
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_100-batch_size_4-learning_rate_0.01-aug_False/ckpt-207.data-00000-of-00001
workspace/models/efficient

### Export model
Before we can export the model, we have to specify which checkpoint we want to export (per default, the last one is used).

In [69]:
new_checkpoint = "ckpt-31"

with open(os.path.join(MODELS_PATH, MODEL_NAME, COMBINATION_NAME, 'checkpoint'), 'r+') as file:
    contents = file.read()

    pattern = r'model_checkpoint_path:\s+"(ckpt-\d+)"'
    match = re.search(pattern, contents)

    # Replace the checkpoint number with the new number
    if match:
        old_checkpoint = match.group(1)
        new_contents = contents.replace(old_checkpoint, new_checkpoint, 1)

        # Move the file pointer to the beginning of the file
        file.seek(0)

        # Overwrite the file with the new contents
        file.write(new_contents)

        # Truncate the file to remove any extra characters
        file.truncate()

In [71]:
!python models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path=workspace/models/{MODEL_NAME}/{COMBINATION_NAME}/pipeline.config \
    --trained_checkpoint_dir=workspace/models/{MODEL_NAME}/{COMBINATION_NAME} \
    --output_directory=workspace/models/{MODEL_NAME}/{COMBINATION_NAME}/output

/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase3GetEPNS_15OpKern